In [1]:
import pandas as pd

In [2]:
# Load the feather file into a pandas dataframe
df_data = pd.read_feather("results_v3.feather")

In [3]:
print(df_data.iloc[0])

index                                                           0
year                                                         2022
title           US military creates space unit in South Korea ...
url             https://www.airforcetimes.com/battlefield-tech...
text_content    SEOUL, South Korea — The U.S. military formall...
Name: 0, dtype: object


---
The *News Labels Classifier* is running on the local machine.  
Text contents are sended to it.  
In response, the *News Labels Classifier* sends the scriptors and their score.

In [4]:
import requests
import json

In [5]:
headers = {
    "Content-Type": "application/json"
}

In [9]:

def dump_labels(descriptors):
    df_labels = pd.DataFrame(columns=['label','score'])
    counter = 0
    for d in descriptors:
        df_labels.loc[counter] = [d['label'], d['score']]
        counter = counter + 1 
    return df_labels

df_results = pd.DataFrame(columns=['article','tags'])
for counter_data in range(0, len(df_data)):
    data = {
        "text" : df_data.loc[counter_data].text_content # first record of the dataframe
    }
    response = requests.post("http://localhost:8000/predict.json", data=json.dumps(data), headers=headers)
    response_text = json.loads(response.text)
    
    # Retrieve the descriptors ( predicted tags with e score)
    descriptors = (response_text['predictions']['allDescriptors'])
    df_tags = dump_labels(descriptors)
    
    # Append to the results
    df_results.loc[counter_data] = [df_data.loc[counter_data].url,df_tags]
print(df_results.info())

# Export dataframe to a feather format file
df_results.to_json('results_tags.json', orient='records')

print("Opération terminée")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5279 entries, 0 to 5278
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   article  5279 non-null   object
 1   tags     5279 non-null   object
dtypes: object(2)
memory usage: 123.7+ KB
None
Opération terminée
